In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
customer_df = pd.read_csv("customers.csv", delimiter=",")
customer_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'customers.csv'

In [ ]:
orders_df = pd.read_csv("orders.csv", delimiter=",")
orders_df.head()

In [ ]:
product_df = pd.read_csv("products.csv", delimiter=",")
product_df.head()

In [ ]:
sales_df = pd.read_csv("sales.csv", delimiter=",")
sales_df.head()

In [ ]:
customer_df.info()

In [ ]:
customer_df.isna().sum()

In [ ]:
print("Jumlah duplikasi : "), customer_df.duplicated().sum()

In [ ]:
customer_df.describe()

In [ ]:
orders_df.info()

In [ ]:
print("Jumlah duplikasi: ", orders_df.duplicated().sum())

orders_df.describe()

In [ ]:
product_df.info()

In [ ]:
print("Jumlah duplikasi : ", product_df.duplicated().sum())

product_df.describe()

In [ ]:
sales_df.info()

In [ ]:
sales_df.isna().sum()

In [ ]:
print("Jumlah duplikasi : ", sales_df.duplicated().sum())

sales_df.describe()

In [ ]:
customer_df.drop_duplicates(inplace=True)

In [ ]:
print("Jumlah duplikasi : ", customer_df.duplicated().sum())

In [ ]:
customer_df[customer_df.gender.isna()]

In [ ]:
customer_df.gender.value_counts()

In [ ]:
customer_df.fillna(value="Prefer not to say", inplace=True)

In [ ]:
customer_df.isna().sum()

In [ ]:
customer_df[customer_df.age == customer_df.age.max()]

In [ ]:
customer_df.age.replace(customer_df.age.max(), 70, inplace=True)

In [ ]:
customer_df[customer_df.age == customer_df.age.max()]

In [ ]:
customer_df.age.replace(customer_df.age.max(), 50, inplace=True)

In [ ]:
customer_df.describe()

In [ ]:
datetime_columns = ["order_date", "delivery_date"]

for column in datetime_columns:
    orders_df[column] = pd.to_datetime(orders_df[column])

In [ ]:
orders_df.info()

In [ ]:
product_df.drop_duplicates(inplace=True)

In [ ]:
print("Jumlah duplikasi : ", product_df.duplicated().sum())

In [ ]:
sales_df[sales_df.total_price.isna()]

In [ ]:
sales_df["total_price"] = sales_df["price_per_unit"] * sales_df["quantity"]

In [ ]:
sales_df.isna().sum()

In [ ]:
customer_df.describe(include="all")

In [ ]:
customer_df.groupby(by='gender').agg({
    "customer_id": "nunique",
    "age": ["max", "min", "mean", "std"]
})

In [ ]:
print(customer_df.groupby(by="city").customer_id.nunique().sort_values(ascending=False))
customer_df.groupby(by="state").customer_id.nunique().sort_values(ascending=False)

In [ ]:
delivery_time = orders_df["delivery_date"] - orders_df["order_date"]
delivery_time = delivery_time.apply(lambda x: x.total_seconds())
orders_df["delivery_time"] = round(delivery_time/86400)

In [ ]:
orders_df.head()

In [ ]:
orders_df.describe(include="all")

In [ ]:
customer_id_in_orders_df =  orders_df.customer_id.tolist()
customer_df["status"] = customer_df["customer_id"].apply(lambda x: "Active" if x in customer_id_in_orders_df else "Non Active")
customer_df.sample(10)

In [ ]:
customer_df.groupby(by="status").customer_id.count()

In [ ]:
order_customer_df = pd.merge(
    left=orders_df,
    right=customer_df,
    how="left",
    left_on="customer_id",
    right_on="customer_id"
)
order_customer_df.head()

In [ ]:
order_customer_df.groupby(by="city").order_id.nunique().sort_values(ascending=False).reset_index().head(10)

In [ ]:
order_customer_df.groupby(by="state").order_id.nunique().sort_values(ascending=False).reset_index().head(10)

In [ ]:
order_customer_df.groupby(by="gender").order_id.nunique().sort_values(ascending=False).reset_index().head(10)

In [ ]:
order_customer_df["age_group"] = order_customer_df.age.apply(lambda x: "Youth" if x<= 24 else ("Senior" if x > 64 else "Adults"))
order_customer_df.groupby(by="age_group").order_id.nunique().sort_values(ascending=False).reset_index().head(10)

In [ ]:
print(product_df.describe(include="all"))
sales_df.describe(include="all")

In [ ]:
product_df.sort_values(by="price", ascending=False)

In [ ]:
product_df.groupby(by="product_type").agg({
    "product_id": "nunique",
    "quantity": "sum",
    "price": ["min", "max"]
})

In [ ]:
product_df.groupby(by="product_name").agg({
    "product_id": "nunique",
    "quantity": "sum",
    "price": ["min", "max"]
})

In [ ]:
sales_product_df = pd.merge(
    left=sales_df,
    right=product_df,
    how="left",
    left_on="product_id",
    right_on="product_id"
)
sales_product_df.head()

In [ ]:
sales_product_df.groupby(by="product_type").agg({
    "sales_id": "nunique",
    "quantity_x": "sum",
    "total_price": "sum"
})

In [ ]:
sales_product_df.groupby(by="product_name").agg({
    "sales_id": "nunique",
    "quantity_x": "sum",
    "total_price": "sum"
}).sort_values(by="total_price", ascending=False)

In [ ]:
all_df = pd.merge(
    left=sales_product_df,
    right=order_customer_df,
    how="left",
    left_on="order_id",
    right_on="order_id"
)
all_df.head()

In [ ]:
all_df.groupby(by=["state", "product_type"]).agg({
    "quantity_x": "sum",
    "total_price": "sum"
})

In [ ]:
all_df.groupby(by=["gender", "product_type"]).agg({
    "quantity_x": "sum",
    "total_price": "sum"
})

In [ ]:
all_df.groupby(by=["age_group", "product_type"]).agg({
    "quantity_x": "sum",
    "total_price": "sum"
})

In [ ]:
monthly_order_df = all_df.resample(rule='M', on='order_date').agg({
    "order_id": "nunique",
    "total_price": "sum"
})